In [1]:
# This directive tells the notebook to add interactive plots in the notebook
#%matplotlib# widget
#%matplotlib
import sys
sys.path.append('../src')  # Add the 'src' directory to the Python path
from model.solver import Solver #bm3_barotropic_model packages
from model.sphere import Sphere
from utils import constants, plotting, parcel_tracking, animations
import ensemble_methods as em

import time
import numpy as np
import random
import xarray as xr

import matplotlib.pyplot as plt    

from tqdm import tqdm
from importlib import reload
from IPython.display import display, clear_output, HTML, FileLink  # Some technical stuff, used to display animation

/home/bertossa/miniconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
st = Sphere(base_state='solid')
st.add_seaice()

params=dict(nu=1e-15, tau = 1/8, diffusion_order=4,dt=1800, ofreq=4, ensemble_size=10,
            forcing_type='rededdy', vort_linear = False, temp_linear=False)


T=7*constants.day2sec

ensemble = em.integrate_ensemble(st=st, T=T, **params)

100%|███████████████████████████████████████████| 10/10 [00:17<00:00,  1.79s/it]


In [3]:
ensemble

<xarray.Dataset>
Dimensions:  (time: 85, y: 64, x: 127, ens_mem: 10)
Coordinates:
  * time     (time) float64 0.0 7.2e+03 1.44e+04 ... 5.976e+05 6.048e+05
  * y        (y) float64 87.86 85.1 82.31 79.53 ... -79.53 -82.31 -85.1 -87.86
  * x        (x) float64 0.0 2.835 5.669 8.504 11.34 ... 348.7 351.5 354.3 357.2
Dimensions without coordinates: ens_mem
Data variables:
    vort     (ens_mem, time, y, x) float64 0.0 0.0 0.0 ... -1.826e-06 -1.863e-06
    vortp    (ens_mem, time, y, x) float64 -3.137e-06 -3.137e-06 ... 1.274e-06
    u        (ens_mem, time, y, x) float64 0.3728 0.3728 ... -0.0263 -0.1289
    v        (ens_mem, time, y, x) float64 -5.725e-08 -2.09e-07 ... 1.809 1.777
    thetap   (ens_mem, time, y, x) float64 0.0 0.0 0.0 ... 0.7845 0.7649 0.7458
    theta    (ens_mem, time, y, x) float64 255.1 255.1 255.1 ... 255.8 255.8

In [8]:
#ensemble=ensemble.assign_coords(ens_mem= range(len(ensemble.ens_mem)))

In [16]:
reload(parcel_tracking)
dsr = ensemble.rename(dict(x='lon',y='lat'))
# Reassign longitude coordinates between -180 and 180E
dsr = dsr.assign_coords(lon=(((dsr.lon + 180) % 360) - 180))
dsr = dsr.sortby('lon')
dsr=dsr.reindex(lat=list(reversed(dsr.lat)))

#x0 should be in form [(lon,lat),]
std = dsr.theta.std('ens_mem').sel(lat=slice(30,90))
sm = std.where(std==std.max(), drop=True).squeeze()
x0 = [(sm.lon.item(0),sm.lat.item(0))]

ts, xs = parcel_tracking.ens_calculate_trajectories(dsr.sel(ens_mem=slice(0,10)), x0, t0 = float(T), rvs=True, tstep_factor=1)


Integrating 1 trajectories for [1209600.].


100%|█████████████████████████████████████████| 167/167 [12:01<00:00,  4.32s/it]

Completed 168 of 168 timesteps.


In [20]:
reload(plotting)
plotting.animate_thetaens(dsr,times = [0,T],xs=xs,ts=ts, tlevs=np.arange(0,9,1))

MovieWriter ffmpeg unavailable; using Pillow instead.


0.0
0.0
0.005988023952095809
0.011976047904191617
0.017964071856287425
0.023952095808383235
0.029940119760479042
0.03592814371257485
0.041916167664670656
0.04790419161676647
0.05389221556886228
0.059880239520958084
0.0658682634730539
0.0718562874251497
0.07784431137724551
0.08383233532934131
0.08982035928143713
0.09580838323353294
0.10179640718562874
0.10778443113772455
0.11377245508982035
0.11976047904191617
0.12574850299401197
0.1317365269461078
0.1377245508982036
0.1437125748502994
0.1497005988023952
0.15568862275449102
0.16167664670658682
0.16766467065868262
0.17365269461077845
0.17964071856287425
0.18562874251497005
0.19161676646706588
0.19760479041916168
0.20359281437125748
0.20958083832335328
0.2155688622754491
0.2215568862275449
0.2275449101796407
0.23353293413173654
0.23952095808383234
0.24550898203592814
0.25149700598802394
0.25748502994011974
0.2634730538922156
0.2694610778443114
0.2754491017964072
0.281437125748503
0.2874251497005988
0.2934131736526946
0.2994011976047904
0.